In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import confusion_matrix,classification_report,precision_recall_fscore_support,accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
import xgboost as xgb

import mlflow

In [10]:
import pickle

with open("tx_class.pkl", "rb") as f:
    tx_class = pickle.load(f)

In [11]:
tx_class.describe()

,CustomerID,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,m6_Revenue,LTVCluster
count,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000
mean,15564.447059,91.649361,2.098721,83.230691,0.112788,1274.397832,0.057033,2.268542,722.131338,0.276215
std,1575.324626,100.365462,1.055131,131.120889,0.329833,1896.155427,0.231936,1.269920,1057.080028,0.526525
min,12346.000000,0.000000,0.000000,1.000000,0.000000,-4287.630000,0.000000,0.000000,-4287.630000,0.000000
25%,14212.250000,16.000000,1.000000,16.250000,0.000000,279.230000,0.000000,1.000000,101.250000,0.000000
50%,15575.000000,49.500000,2.000000,40.000000,0.000000,618.040000,0.000000,2.000000,358.585000,0.000000
75%,16913.750000,144.000000,3.000000,98.000000,0.000000,1483.497500,0.000000,3.000000,917.290000,0.000000
max,18287.000000,373.000000,3.000000,2782.000000,2.000000,21535.900000,1.000000,6.000000,8432.680000,2.000000


In [12]:
#calculate and show correlations
corr_matrix = tx_class.corr()
corr_matrix['LTVCluster'].sort_values(ascending=False)

LTVCluster            1.000000
m6_Revenue            0.878757
Revenue               0.776924
RevenueCluster        0.608345
Frequency             0.568411
OverallScore          0.540214
FrequencyCluster      0.515850
Segment_High-Value    0.496705
RecencyCluster        0.355204
Segment_Mid-Value     0.188219
CustomerID           -0.027388
Recency              -0.346686
Segment_Low-Value    -0.377251
Name: LTVCluster, dtype: float64

In [13]:
#create X and y, X will be feature set and y is the label - LTV
X = tx_class.drop(['LTVCluster','m6_Revenue'],axis=1)
y = tx_class['LTVCluster']

#split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=56)

## Models

Since our LTV Clusters are 3 types, high LTV, mid LTV and low LTV; we will perform multi class classification.

In [14]:
mlflow.set_experiment("CLTV")
mlflow.sklearn.autolog(disable=True)

2023/07/25 10:13:32 INFO mlflow.tracking.fluent: Experiment with name 'CLTV' does not exist. Creating a new experiment.


### 1. XGBoost 

In [15]:
params = {
        "max_depth": 5,
        "learning_rate":0.1}
parsuf = '_'.join([key.replace('_','')+str(val).replace('.','') for key,val in params.items()])
print(parsuf)

maxdepth5_learningrate01


In [17]:
basemodelname = "xgboost"
with mlflow.start_run(run_name=basemodelname):
    params = {
        "max_depth": 5,
        "learning_rate":0.1}
    parsuf = '_'.join([key.replace('_','')+str(val).replace('.','') for key,val in params.items()])
    modelname=f"{basemodelname}_{parsuf}"
    mlflow.set_tag("model_name", modelname)
    mlflow.log_params(params)
    
    ltv_xgb = xgb.XGBClassifier(
        max_depth=params['max_depth'], 
        learning_rate=params['learning_rate'],
        n_jobs=-1
    ).fit(X_train, y_train)

    print('Accuracy of XGB classifier on training set: {:.2f}'
           .format(ltv_xgb.score(X_train, y_train)))
    print('Accuracy of XGB classifier on test set: {:.2f}'
           .format(ltv_xgb_model.score(X_test[X_train.columns], y_test)))

    y_pred = ltv_xgb_model.predict(X_test)
    # clfreport = classification_report(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    prf1s = precision_recall_fscore_support(y_test, y_pred)
    #print(f"precision:{prf1s[0]}\nrecall:{prf1s[1]}\nf1-score:{prf1s[2]}\naccuracy:{acc}")
    mlflow.log_metric('test_acc', acc)
    mlflow.sklearn.log_model(ltv_xgb, "ltv_xgb")

NameError: name 'ltv_xgb_model' is not defined